# Azure AI سرچ کے ساتھ NVIDIA NIM اور LlamaIndex انضمام

اس نوٹ بک میں، ہم NVIDIA کے AI ماڈلز اور LlamaIndex کا استعمال کرتے ہوئے ایک طاقتور Retrieval-Augmented Generation (RAG) پائپ لائن بنانے کا مظاہرہ کریں گے۔ ہم NVIDIA کے LLMs اور embeddings کو استعمال کریں گے، انہیں Azure AI سرچ کے ساتھ بطور ویکٹر اسٹور ضم کریں گے، اور RAG کے ذریعے سرچ کی کوالٹی اور کارکردگی کو بہتر بنائیں گے۔

## فوائد
- **اسکیل ایبلٹی**: NVIDIA کے بڑے زبان ماڈلز اور Azure AI سرچ کا استعمال کرتے ہوئے اسکیل ایبل اور مؤثر ریٹریول حاصل کریں۔
- **لاگت کی بچت**: مؤثر ویکٹر اسٹوریج اور ہائبرڈ سرچ تکنیک کے ذریعے سرچ اور ریٹریول کو بہتر بنائیں۔
- **اعلی کارکردگی**: طاقتور LLMs کو ویکٹرائزڈ سرچ کے ساتھ ملا کر تیز اور زیادہ درست جوابات حاصل کریں۔
- **کوالٹی**: متعلقہ دستاویزات کے ذریعے LLM جوابات کو بنیاد فراہم کرتے ہوئے سرچ کی کوالٹی برقرار رکھیں۔

## ضروریات
- 🐍 Python 3.9 یا اس سے زیادہ
- 🔗 [Azure AI سرچ سروس](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API Key تاکہ NVIDIA کے LLMs اور Embeddings تک رسائی حاصل کی جا سکے NVIDIA NIM مائیکرو سروسز کے ذریعے

## شامل کردہ خصوصیات
- ✅ NVIDIA LLM انضمام (ہم استعمال کریں گے [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA Embeddings (ہم استعمال کریں گے [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Azure AI سرچ کے جدید ریٹریول موڈز
- ✅ دستاویزات کی انڈیکسنگ LlamaIndex کے ساتھ
- ✅ RAG Azure AI سرچ اور LlamaIndex کے ساتھ NVIDIA LLMs کا استعمال کرتے ہوئے

چلیں شروع کرتے ہیں!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## تنصیب اور ضروریات  
پائتھون ورژن >3.10 استعمال کرتے ہوئے ایک پائتھون ماحول بنائیں۔

## شروعات کریں!  


NVIDIA AI Foundation ماڈلز استعمال کرنے کے لیے آپ کو `NVIDIA_API_KEY` کی ضرورت ہوگی:
1) [NVIDIA](https://build.nvidia.com/explore/discover) کے ساتھ ایک مفت اکاؤنٹ بنائیں۔
2) اپنی پسند کے ماڈل پر کلک کریں۔
3) Input کے تحت، Python ٹیب منتخب کریں، پھر **Get API Key** پر کلک کریں اور **Generate Key** پر کلک کریں۔
4) بنائی گئی key کو NVIDIA_API_KEY کے طور پر کاپی کریں اور محفوظ کریں۔ اس کے بعد، آپ کو endpoints تک رسائی حاصل ہوگی۔


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## ایل ایل ایم اور ایمبیڈنگ کے ساتھ RAG مثال
### 1) ایل ایل ایم کو شروع کریں
`llama-index-llms-nvidia`، جسے NVIDIA کا ایل ایل ایم کنیکٹر بھی کہا جاتا ہے، آپ کو NVIDIA API کیٹلاگ میں دستیاب ماڈلز سے جڑنے اور ان سے مواد تخلیق کرنے کی اجازت دیتا ہے۔ چیٹ کمپلیشن ماڈلز کی فہرست کے لیے یہاں دیکھیں: https://build.nvidia.com/search?term=Text-to-Text

یہاں ہم **mixtral-8x7b-instruct-v0.1** استعمال کریں گے۔


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) ایمبیڈنگ کو شروع کریں

`llama-index-embeddings-nvidia`، جسے NVIDIA کے ایمبیڈنگ کنیکٹر کے نام سے بھی جانا جاتا ہے، آپ کو NVIDIA API کیٹلاگ میں دستیاب ماڈلز سے جڑنے اور ان سے جنریشن کرنے کی اجازت دیتا ہے۔ ہم نے ایمبیڈنگ ماڈل کے طور پر `nvidia/nv-embedqa-e5-v5` کا انتخاب کیا۔ ٹیکسٹ ایمبیڈنگ ماڈلز کی فہرست یہاں دیکھیں: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) ایک Azure AI سرچ ویکٹر اسٹور بنائیں


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

### دستاویزات کو لوڈ کریں، ٹکڑوں میں تقسیم کریں، اور اپلوڈ کریں


In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) اپنے ڈیٹا پر سوالات پوچھنے کے لیے ایک کوئری انجن بنائیں

یہاں ایک کوئری ہے جو Azure AI Search میں خالص ویکٹر تلاش کا استعمال کرتی ہے اور ہمارے LLM (Phi-3.5-MOE) کے جواب کو بنیاد فراہم کرتی ہے۔


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

یہاں Azure AI سرچ میں ہائبرڈ سرچ استعمال کرنے کی ایک استفسار ہے۔


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### ویکٹر سرچ تجزیہ  
ایل ایل ایم کا جواب روس کی اسٹاک مارکیٹ کے حوالے سے ماخذ متن میں ذکر کردہ اہم اقتصادی نتائج کو درست طریقے سے بیان کرتا ہے۔ خاص طور پر، یہ بتاتا ہے کہ روسی اسٹاک مارکیٹ میں نمایاں کمی ہوئی، جس سے اس کی قدر کا 40% نقصان ہوا، اور جاری صورتحال کی وجہ سے تجارت معطل کر دی گئی۔ یہ جواب ماخذ میں فراہم کردہ معلومات کے ساتھ اچھی طرح مطابقت رکھتا ہے، جو ظاہر کرتا ہے کہ ایل ایل ایم نے اسٹاک مارکیٹ پر اثرات کے متعلق اہم تفصیلات کو درست طریقے سے پہچانا اور خلاصہ کیا۔

#### ماخذ نوڈز پر تبصرہ  
ماخذ نوڈز بین الاقوامی پابندیوں کی وجہ سے روس کو درپیش اقتصادی نتائج کی تفصیلی وضاحت فراہم کرتے ہیں۔ متن میں یہ بات نمایاں کی گئی ہے کہ روسی اسٹاک مارکیٹ نے اپنی قدر کا 40% کھو دیا، اور تجارت معطل کر دی گئی۔ اس کے علاوہ، دیگر اقتصادی اثرات کا بھی ذکر کیا گیا ہے، جیسے روبل کی قدر میں کمی اور روسی معیشت کی وسیع پیمانے پر تنہائی۔ ایل ایل ایم کے جواب نے ان نوڈز سے اہم نکات کو مؤثر طریقے سے اخذ کیا، خاص طور پر اسٹاک مارکیٹ کے اثرات پر توجہ مرکوز کرتے ہوئے، جیسا کہ سوال میں درخواست کی گئی تھی۔


اب آئیے ایک ایسی مثال دیکھتے ہیں جہاں ہائبرڈ سرچ ایک مضبوط جواب فراہم نہیں کرتی:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### ہائبرڈ سرچ: ایل ایل ایم ردعمل کا تجزیہ
ہائبرڈ سرچ کی مثال میں ایل ایل ایم کا ردعمل ظاہر کرتا ہے کہ فراہم کردہ سیاق و سباق میں روس کے یوکرین پر حملے کی صحیح تاریخ کا ذکر نہیں ہے۔ یہ ردعمل اس بات کی نشاندہی کرتا ہے کہ ایل ایل ایم دستیاب معلومات کو استعمال کر رہا ہے لیکن متن میں موجود تفصیلات کی کمی کو تسلیم کرتا ہے۔

یہ ردعمل درست ہے کیونکہ یہ پہچانتا ہے کہ سیاق و سباق میں روس کی جارحیت سے متعلق واقعات کا ذکر ہے لیکن حملے کی مخصوص تاریخ کا ذکر نہیں ہے۔ یہ ایل ایل ایم کی صلاحیت کو ظاہر کرتا ہے کہ وہ فراہم کردہ معلومات کو سمجھ سکتا ہے اور مواد میں موجود خلا کو پہچان سکتا ہے۔ ایل ایل ایم مؤثر طریقے سے صارف کو بیرونی ذرائع یا تاریخی ریکارڈز سے صحیح تاریخ تلاش کرنے کی ترغیب دیتا ہے، اور جب معلومات مکمل نہ ہوں تو محتاط رویہ اپناتا ہے۔

### سورس نوڈز کا تجزیہ
ہائبرڈ سرچ کی مثال میں سورس نوڈز میں ایک تقریر کے اقتباسات شامل ہیں جو یوکرین میں روس کی کارروائیوں کے جواب میں امریکہ کے ردعمل پر بات کرتے ہیں۔ یہ نوڈز وسیع تر جغرافیائی سیاسی اثرات اور حملے کے جواب میں امریکہ اور اس کے اتحادیوں کے اقدامات پر زور دیتے ہیں، لیکن حملے کی مخصوص تاریخ کا ذکر نہیں کرتے۔ یہ ایل ایل ایم کے ردعمل کے ساتھ ہم آہنگ ہے، جو درست طور پر پہچانتا ہے کہ سیاق و سباق میں صحیح تاریخ کی معلومات موجود نہیں ہیں۔


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### ہائبرڈ w/Reranking: LLM جواب کا تجزیہ
ہائبرڈ w/Reranking کی مثال میں، LLM جواب اضافی سیاق و سباق فراہم کرتا ہے، یہ بتاتے ہوئے کہ واقعہ تقریر دیے جانے سے چھ دن پہلے ہوا تھا۔ یہ ظاہر کرتا ہے کہ LLM تقریر کے وقت کی بنیاد پر حملے کی تاریخ کا اندازہ لگا سکتا ہے، حالانکہ اسے درستگی کے لیے تقریر کی صحیح تاریخ جاننے کی ضرورت ہوتی ہے۔

یہ جواب سیاق و سباق کے اشارے استعمال کرنے کی بہتر صلاحیت کو ظاہر کرتا ہے تاکہ زیادہ معلوماتی جواب فراہم کیا جا سکے۔ یہ reranking کے فائدے کو اجاگر کرتا ہے، جہاں LLM زیادہ متعلقہ معلومات تک رسائی حاصل کر سکتا ہے اور ترجیح دے سکتا ہے تاکہ مطلوبہ تفصیل (یعنی حملے کی تاریخ) کے قریب تر اندازہ فراہم کیا جا سکے۔

### سورس نوڈز کا تجزیہ
اس مثال میں سورس نوڈز میں روس کے حملے کے وقت کے حوالے شامل ہیں، خاص طور پر یہ ذکر کرتے ہوئے کہ یہ تقریر سے چھ دن پہلے ہوا تھا۔ حالانکہ صحیح تاریخ واضح طور پر بیان نہیں کی گئی، نوڈز عارضی سیاق و سباق فراہم کرتے ہیں جو LLM کو زیادہ باریک بینی سے جواب دینے کی اجازت دیتا ہے۔ اس تفصیل کا شامل ہونا یہ ظاہر کرتا ہے کہ reranking کس طرح فراہم کردہ سیاق و سباق سے معلومات نکالنے اور اندازہ لگانے کی LLM کی صلاحیت کو بہتر بنا سکتا ہے، جس کے نتیجے میں زیادہ درست اور معلوماتی جواب حاصل ہوتا ہے۔


**نوٹ:**  
اس نوٹ بک میں، ہم نے NVIDIA API کیٹلاگ سے NVIDIA NIM مائیکرو سروسز استعمال کی ہیں۔  
اوپر دی گئی APIs، `NVIDIA (llms)`, `NVIDIAEmbedding`, اور [Azure AI Search Semantic Hybrid Retrieval (بلٹ ان رینکنگ)](https://learn.microsoft.com/azure/search/semantic-search-overview)۔  
نوٹ کریں کہ یہ APIs خود میزبان مائیکرو سروسز کو بھی سپورٹ کر سکتی ہیں۔  

**مثال:**  
```python  
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")  



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
